In [1]:
%run __init__.py

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import news_api
from nlp import polarity_score, text_tonkenize
from scrapy import Scrapy
from news_api import News_api

In [29]:
with open ("./newsapi/newsapi_key1.pkl", 'rb') as nak:
    newsapi_key = pickle.load(nak)
    nak.close()

In [30]:
apple_obj = News_api(newsapi_key, "2021-05-17","2021-05-21")

In [31]:
sources = ['abc-news', 'business-insider', 'financial-post', 'google-news', 'reuters',
           'nbc-news', 'techcrunch', 'wired', 'the-wall-street-journal']

In [32]:
apple_news = apple_obj.get_news(["apple"], sources=sources)

In [75]:
apple_newsapi_df = pd.DataFrame(apple_news['apple'])
apple_newsapi_df.head(3)

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': 'abc-news', 'name': 'ABC News'}",The Associated Press,How major US stock indexes fared Friday,Stocks ended a wobbly day with mixed results o...,https://abcnews.go.com/Business/wireStory/majo...,null,2021-05-21T20:18:28Z,Stocks ended a wobbly day with mixed results o...
1,"{'id': 'abc-news', 'name': 'ABC News'}",The Associated Press,Lady Gaga says rape at 19 led to a 'total psyc...,Lady Gaga says she got pregnant at 19 after be...,https://abcnews.go.com/Entertainment/wireStory...,https://s.abcnews.com/images/Lifestyle/WireAP_...,2021-05-21T14:26:30Z,Lady Gaga says she got pregnant at 19 after be...
2,"{'id': 'abc-news', 'name': 'ABC News'}",ELAINE KURTENBACH AP Business Writer,Asian stocks mixed after Wall St rebound break...,Asian shares are mixed after a rebound on Wall...,https://abcnews.go.com/Business/wireStory/asia...,https://s.abcnews.com/images/Business/WireAP_6...,2021-05-21T06:48:47Z,BANGKOK -- Asian shares were mixed Friday afte...


In [40]:
apple_newsapi_df['url'][0]

'https://abcnews.go.com/Business/wireStory/major-us-stock-indexes-fared-friday-77835436'

In [174]:
apple_newsapi_df.to_csv("./data/apple_newsapi_raw.csv")

In [56]:
articles = Scrapy.scrap(apple_newsapi_df.url.values.tolist())

In [57]:
apple_newsapi_articles_df = pd.Series(articles, name='articles')
apple_newsapi_articles_df[:5]

0    [Stocks ended a wobbly day with mixed results ...
1    [(76, https://techcrunch.com/2021/05/21/daily-...
Name: articles, dtype: object

In [192]:
apple_newsapi_articles_df = text_tonkenize(apple_newsapi_articles_df)

In [195]:
apple_newsapi_scores_df = polarity_score(apple_newsapi_articles_df)

In [202]:
apple_newsapi_scores_df.head()

,text,negative,neutral,positive,compound,sentiment
0,stocks ended a wobbly day with mixed results o...,0.060,0.889,0.052,-0.1280,negative
1,lady gaga says she got pregnant at after being...,0.142,0.838,0.020,-0.9834,negative
2,stocks ended a wobbly day with mixed results o...,0.050,0.845,0.105,0.9750,positive
3,apple ceo tim cook will take the witness stand...,0.049,0.841,0.110,0.9941,positive
4,apple ceo tim cook will take the witness stand...,0.049,0.841,0.110,0.9941,positive


## Data Scrubbing

In [234]:
apple_newsapi_df = pd.concat([apple_newsapi_df, apple_newsapi_scores_df],
                                      axis=1, ignore_index=False, sort=False).reindex()

In [252]:
apple_newsapi_df.rename(columns={"publishedAt":"datetime"}, inplace=True)

In [260]:
apple_newsapi_df['datetime'] = pd.to_datetime(apple_newsapi_df['datetime'], infer_datetime_format=True)

In [264]:
apple_newsapi_df['source'] = apple_newsapi_df['source'].apply(lambda s: s['name'] if s['name']!=None else s['id'] if s['id']!=None else s)

In [771]:
# we got just 27 missing scraped articles, those will be droped
apple_newsapi_df.dropna(inplace=True)

In [267]:
apple_newsapi_df['word_count'] = apple_newsapi_df['text'].apply(lambda x: len(x.split()))

In [270]:
apple_newsapi_df.drop(apple_newsapi_df[(apple_newsapi_df['word_count']==0)].index, inplace=True)

In [274]:
apple_newsapi_df.to_csv("./data/apple_newsapi_df.csv")

In [277]:
apple_newsapi_df.head(3)

,source,author,title,description,url,urlToImage,datetime,content,text,negative,neutral,positive,compound,sentiment,word_count
0,ABC News,The Associated Press,How major US stock indexes fared Friday,Stocks ended a wobbly day with mixed results o...,https://abcnews.go.com/Business/wireStory/majo...,null,2021-05-21 20:18:28,Stocks ended a wobbly day with mixed results o...,stocks ended a wobbly day with mixed results o...,0.060,0.889,0.052,-0.1280,negative,195
1,ABC News,The Associated Press,Lady Gaga says rape at 19 led to a 'total psyc...,Lady Gaga says she got pregnant at 19 after be...,https://abcnews.go.com/Entertainment/wireStory...,https://s.abcnews.com/images/Lifestyle/WireAP_...,2021-05-21 14:26:30,Lady Gaga says she got pregnant at 19 after be...,lady gaga says she got pregnant at after being...,0.142,0.838,0.020,-0.9834,negative,221
2,ABC News,ELAINE KURTENBACH AP Business Writer,Asian stocks mixed after Wall St rebound break...,Asian shares are mixed after a rebound on Wall...,https://abcnews.go.com/Business/wireStory/asia...,https://s.abcnews.com/images/Business/WireAP_6...,2021-05-21 06:48:47,BANGKOK -- Asian shares were mixed Friday afte...,stocks ended a wobbly day with mixed results o...,0.050,0.845,0.105,0.9750,positive,539


In [ ]:
apple_newsapi_df.set_index(keys='datetime', inplace=True)

In [ ]:
apple_newsapi_df['day'] = apple_newsapi_df.index.date